In [ ]:
# =============================================================================
#   Load the required packages
# =============================================================================
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  Dropout, Conv2D, MaxPooling2D, Flatten, Dense
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.metrics import classification_report, accuracy_score,confusion_matrix
from astropy.stats import sigma_clipped_stats

import matplotlib.pyplot as plt
import numpy as np
import os
import splitfolders
import shutil
import cv2
import time;

from tensorflow.keras.layers import Input, Dense, concatenate, Activation,LSTM,Bidirectional
from tensorflow.keras.models import Model

import glob
from PIL import Image
import tensorflow as tf

In [ ]:
# =============================================================================
#   Define a global variables
# =============================================================================
seedNumbers = [1,2,3,4,5]
batchSize = 32
sigma = 3.0
activation_conv = 'relu'
presnost = []

In [ ]:
# =============================================================================
#   Define a path
# ============================================================================
path_to_dataset = 'path to dataset'
splited_dir_path = 'path to sploted dataset'
path_to_model = 'path to saved model'
temp_fold = 'path to temp folder'
test_dir_name = 'path to test folder'
train_dir_name = 'path to train folder'

path_to_unprocessed = 'path to original images - for one class - for example - /data/COMP/'
transformed_data_target ='path to transformed data - /data/COMP/'

In [ ]:
# =============================================================================
#   Function for plot loss and accuracy during training
#   Input: classifier
#   Output: graphs of accuracy and loss
# =============================================================================
def trainingLoss(history):
    plt.style.use('ggplot')
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('')
    plt.ylabel('Accuracy')
    plt.xlabel('Epochs')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
    plt.style.use('ggplot')
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('')
    plt.ylabel('Loss')
    plt.xlabel('Epochs')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

In [ ]:
# =============================================================================
#   Function for evalute test set 
#   Input: test dataset, classifier
#   Output: metrics 
# =============================================================================
def metrics(test_set, classifier):
    test_set.reset()
    numfiles = sum([len(files) for r, d, files in os.walk(test_dir_name)])
    print(numfiles)
    
    Y_pred = classifier.predict(test_set)
    print(Y_pred.shape)
    
    classes = test_set.classes[test_set.index_array]
    print('Classes', classes)
    
    y_pred = np.argmax(Y_pred, axis=-1)
    print('Prediction', y_pred)
    
    accuracy = accuracy_score(y_pred, classes)
    print("Accuracy score: ", accuracy)
    
    print(confusion_matrix(classes, y_pred))
    print(classification_report(classes, y_pred))
    
    return accuracy

In [ ]:
# =============================================================================
#   Function for initialize CNN model
#   Input: 
#   Output: classifier
# =============================================================================
def initializeModel():
    inputs = Input(shape=(64,64,1))
    a = tf.keras.layers.Conv2D(128, kernel_size = (3,3), padding = "valid", strides=(2, 2), activation=activation_conv)(inputs)
    a = tf.keras.layers.Flatten()(a)

    b = tf.keras.layers.Conv2D(64, kernel_size = (4,4), padding = "valid", activation=activation_conv)(inputs)
    b = tf.keras.layers.MaxPooling2D(pool_size=(2,2))(b)
    b = tf.keras.layers.Flatten()(b)

    c = tf.keras.layers.Conv2D(64, kernel_size = (2,2), padding = "valid", activation=activation_conv)(inputs)
    c = tf.keras.layers.MaxPooling2D(pool_size=(2,2))(c)
    c = tf.keras.layers.Flatten()(c)
    
    x = tf.keras.layers.concatenate([a, b, c])
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    output = tf.keras.layers.Dense(units = 4, activation='softmax')(x)

    classifier = tf.keras.Model(inputs=inputs, outputs=output)
    classifier.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    classifier.summary()

    return classifier

### TEST SET 1
- input dataset with rotations (90, 180, 270)

In [ ]:
# =============================================================================
#   MAIN FUNCTION
# =============================================================================
def main():   
    # =============================================================================
    #  Call the data preprocessing function if data needs to be preprocessed
    # =============================================================================
    # preproces_img(path_to_unprocessed, path_to_dataset)
    # Iteration by seeds
    for i in seedNumbers:
        
        # Random distribution of a dataset by seed
        splitfolders.ratio(path_to_dataset, output=splited_dir_path, seed=i, ratio=(.8, .2)) 

        # rotate image in train set
        os.chdir("/train/COMP") 
        for file in glob.glob("*.jpg"):
            for angle in (90, 180, 270):
                image = Image.open(file)
                image_rot = image.rotate(angle)
                image_rot.save(file.replace(".jpg", "_r{0}.jpg".format(angle)))
        os.chdir("/train/FRI")
        for file in glob.glob("*.jpg"):
            for angle in (90, 180, 270):
                image = Image.open(file)
                image_rot = image.rotate(angle)
                image_rot.save(file.replace(".jpg", "_r{0}.jpg".format(angle)))
        os.chdir("/train/FRII")
        for file in glob.glob("*.jpg"):
            for angle in (90, 180, 270):
                image = Image.open(file)
                image_rot = image.rotate(angle)
                image_rot.save(file.replace(".jpg", "_r{0}.jpg".format(angle)))
        os.chdir("/train/BENT")
        for file in glob.glob("*.jpg"):
            for angle in (90, 180, 270):
                image = Image.open(file)
                image_rot = image.rotate(angle)
                image_rot.save(file.replace(".jpg", "_r{0}.jpg".format(angle)))
                
        #  initialize model
        classifier = initializeModel()
        
        # plot model
        plot_model(classifier, to_file='model.png', show_shapes=True, show_layer_names=True)
        numfiles = sum([len(files) for r, d, files in os.walk(path_to_dataset)])
        
        # save models
        classifier.save(temp_fold + 'first_train.hdf5')
        classifier.save(temp_fold + 'second_train.hdf5')
        
        # chcekpoint to save best model        
        checkpoint_callback1 = ModelCheckpoint(temp_fold + 'first_train.hdf5', 
                                               monitor = 'val_loss', 
                                               verbose = 1, 
                                               save_best_only = True, 
                                               mode = 'min')
        checkpoint_callback2 = ModelCheckpoint(temp_fold + 'second_train.hdf5', 
                                               monitor = 'val_loss',
                                               verbose = 1,
                                               save_best_only = True, 
                                               mode = 'min')
        
        # data augumentations techniques
        train_datagen_1 = ImageDataGenerator(brightness_range = [1,1.5], 
                                             validation_split = 0.1)
        train_datagen_2 = ImageDataGenerator(horizontal_flip = True, 
                                             vertical_flip = True, 
                                             validation_split = 0.1)
        
        training_set_1 = train_datagen_1.flow_from_directory(train_dir_name, 
                                                             target_size = (64,64), 
                                                             class_mode = 'categorical', 
                                                             subset = 'training', 
                                                             color_mode ='grayscale')
        
        training_set_2 = train_datagen_2.flow_from_directory(train_dir_name, 
                                                             target_size = (64,64), 
                                                             class_mode = 'categorical', 
                                                             subset = 'training', 
                                                             color_mode = 'grayscale')
        
        validation_set_1 = train_datagen_1.flow_from_directory(train_dir_name, 
                                                               target_size = (64,64),
                                                               batch_size = batchSize, 
                                                               class_mode = 'categorical',
                                                               subset = 'validation', 
                                                               color_mode = 'grayscale')
        
        validation_set_2 = train_datagen_2.flow_from_directory(train_dir_name,
                                                               target_size = (64,64), 
                                                               batch_size = batchSize, 
                                                               class_mode = 'categorical', 
                                                               subset = 'validation', 
                                                               color_mode = 'grayscale')

        test_datagen = ImageDataGenerator()
        test_set = test_datagen.flow_from_directory(test_dir_name,
                                                    target_size = (64,64), 
                                                    class_mode = 'categorical', 
                                                    color_mode = 'grayscale')
        
    # =============================================================================
    #   train model
    # =============================================================================
        start = time.time()
        history=classifier.fit(training_set_1, validation_data=validation_set_1, epochs=50, callbacks= [checkpoint_callback1])  
        classifier.load_weights(temp_fold + "first_train.hdf5")
        trainingLoss(history)
        metrics(test_set, classifier)
        
        history=classifier.fit(training_set_2, validation_data=validation_set_2, epochs=50, callbacks= [checkpoint_callback2])  
        classifier.load_weights(temp_fold + "second_train.hdf5")
        end = time.time()
        hours, rem = divmod(end-start, 3600)
        minutes, seconds = divmod(rem, 60)
        print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
        
    # =============================================================================
    #   predict and evalute test set
    # =============================================================================
        trainingLoss(history)
        start = time.time()
        acc_av= metrics(test_set, classifier)
        end = time.time()
        hours, rem = divmod(end-start, 3600)
        minutes, seconds = divmod(rem, 60)
        print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
        classifier.save(path_to_model + str(acc_av) +'model.hdf5')
        presnost.append(metrics(test_set, classifier))
        
    # =============================================================================
    #   remove split folders
    # =============================================================================
        shutil.rmtree(splited_dir_path)
        print('Removed')
main()

### TEST SET 2
- input dataset without rotation

In [ ]:
# =============================================================================
#   MAIN FUNCTION
# =============================================================================
def main():   
    # =============================================================================
    #  Call the data preprocessing function if data needs to be preprocessed
    # =============================================================================
    #  preproces_img(path_to_unprocessed, path_to_dataset)
    # Iteration by seeds
    for i in seedNumbers:
        
        # Random distribution of a dataset by seed
        split_folders.ratio(path_to_dataset, output=splited_dir_path, seed=i, ratio=(.8, .2)) 
        classifier = initializeModel()
        
        # plot model
        plot_model(classifier, to_file='model.png', show_shapes=True, show_layer_names=True)
        numfiles = sum([len(files) for r, d, files in os.walk(path_to_dataset)])
        
        # save models
        classifier.save(temp_fold + 'first_train.hdf5')
        classifier.save(temp_fold + 'second_train.hdf5')
        
        # chcekpoint to save best model
        checkpoint_callback1 = ModelCheckpoint(temp_fold + 'first_train.hdf5', 
                                               monitor = 'val_loss', 
                                               verbose = 1, 
                                               save_best_only = True,
                                               mode = 'min')
        checkpoint_callback2 = ModelCheckpoint(temp_fold + 'second_train.hdf5',
                                               monitor = 'val_loss', 
                                               verbose = 1, 
                                               save_best_only = True, 
                                               mode = 'min')
       
        # data augumentations techniques
        train_datagen_1 = ImageDataGenerator(brightness_range = [1,1.5], 
                                             validation_split = 0.1)
        train_datagen_2 = ImageDataGenerator(horizontal_flip = True, 
                                             vertical_flip = True, 
                                             validation_split = 0.1)
        
        training_set_1 = train_datagen_1.flow_from_directory(train_dir_name, 
                                                             target_size = (64,64), 
                                                             class_mode = 'categorical', 
                                                             subset = 'training', 
                                                             color_mode ='grayscale')
        
        training_set_2 = train_datagen_2.flow_from_directory(train_dir_name, 
                                                             target_size = (64,64), 
                                                             class_mode = 'categorical', 
                                                             subset = 'training', 
                                                             color_mode = 'grayscale')
        
        validation_set_1 = train_datagen_1.flow_from_directory(train_dir_name, 
                                                               target_size = (64,64),
                                                               batch_size = batchSize, 
                                                               class_mode = 'categorical',
                                                               subset = 'validation', 
                                                               color_mode = 'grayscale')
        
        validation_set_2 = train_datagen_2.flow_from_directory(train_dir_name,
                                                               target_size = (64,64), 
                                                               batch_size = batchSize, 
                                                               class_mode = 'categorical', 
                                                               subset = 'validation', 
                                                               color_mode = 'grayscale')

        test_datagen = ImageDataGenerator()
        test_set = test_datagen.flow_from_directory(test_dir_name,
                                                    target_size = (64,64), 
                                                    class_mode = 'categorical', 
                                                    color_mode = 'grayscale')
    
    # =============================================================================
    #   train model
    # =============================================================================
        start = time.time()
        history=classifier.fit(training_set_1, validation_data=validation_set_1, epochs=50, callbacks=[checkpoint_callback1])  
        classifier.load_weights(temp_fold + "first_train.hdf5")
        trainingLoss(history)
        metrics(test_set, classifier)
        
        history=classifier.fit(training_set_2, validation_data=validation_set_2, epochs=50, callbacks=[checkpoint_callback2])  
        classifier.load_weights(temp_fold + "second_train.hdf5")
        end = time.time()
        hours, rem = divmod(end-start, 3600)
        minutes, seconds = divmod(rem, 60)
        print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
        
    # =============================================================================
    #   predict and evalute test set
    # =============================================================================
        trainingLoss(history)
        start = time.time()
        acc_av= metrics(test_set, classifier)
        end = time.time()
        hours, rem = divmod(end-start, 3600)
        minutes, seconds = divmod(rem, 60)
        print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
        classifier.save(path_to_model + str(acc_av) +'model.hdf5')
        presnost.append(metrics(test_set, classifier))
        
    # =============================================================================
    #   remove split folders
    # =============================================================================
        shutil.rmtree(splited_dir_path)
        print('Removed')
    
main()